# Generating mock data to populate the database

This notebook creates one pandas dataframe for each table of the database, conforming to the schema as in init.sql script and db_models.py module.

Libraries for data processing and random data generation:

In [40]:
import numpy as np
import pandas as pd
import random
from mimesis import Person, Address, Datetime, Text
from mimesis.locales import Locale

## Users

For the Users table, create 12 users with real-looking information for Germany.

In [41]:
dict_users = {
    "id" : [], 
    "username": [],
    "password_hash": [],
    "email": [],
    "first_name": [],
    "last_name": [],
    "date_of_birth": [],
    "street_address": [],
    "city": [],
    "postal_code": [],
    "registration_datetime": [],
    "user_type": [],
}

N_USERS = 12

for id in range(1, N_USERS + 1):
    person_gen = Person(Locale.DE, seed = 42 + id)
    address_gen = Address(Locale.DE, seed = 42 + id)
    date_gen = Datetime(Locale.DE, seed = 42 + id)

    dict_users["id"].append(id)
    dict_users["username"].append(person_gen.username())
    dict_users["password_hash"].append(person_gen.password(hashed = True))
    dict_users["email"].append(person_gen.email())
    dict_users["first_name"].append(person_gen.first_name())
    dict_users["last_name"].append(person_gen.last_name())
    dict_users["date_of_birth"].append(person_gen.birthdate(min_year = 1960, max_year = 2000).strftime("%Y-%m-%d"))
    dict_users["street_address"].append(address_gen.address())
    dict_users["city"].append(address_gen.city())
    dict_users["postal_code"].append(address_gen.postal_code())
    dict_users["registration_datetime"].append(date_gen.datetime(start = 2023, end = 2024).strftime("%Y-%m-%d %H:%M:%S"))
    dict_users["user_type"].append("test")

df_users = pd.DataFrame(dict_users)

## Coffee machines

For the CoffeeMachines table, enter values manually for 6 machines (info not necessarily accurate).

In [42]:
df_coffee_machines = pd.DataFrame(
    columns = [
        "id", 
        "manufacturer",
        "model_name",
        "model_name_add",
        "model_specification",
        "product_identifier",
        "pump_pressure_bar",
        "pump_type",
        "water_temp_control",
        "pid_control",
        "boiler_type",
        "portafilter_diam_mm"
    ], 
    data = [
        (1, 'DeLonghi', 'Dedica', '', 'Black', 'EC685.BK', 15, 'vibrating', 'no', '', 'single boiler', 51),
        (2, 'Sage/Breville', 'Barista', 'Pro', 'Brushed Stainless Steel', 'SES878BSS4EEU1', 15, 'vibrating', 'yes', 'automatic', 'dual boiler', 54),
        (3, 'Sage/Breville', 'Oracle', 'Touch', 'Matte Black', 'SES990BSS4EEU1', 9, 'vibrating', 'yes', 'programmable', 'dual boiler', 58),
        (4, 'Lelit', 'Bianca', '', 'Steel', 'LE-PL162T V3', 10, 'rotary pump', 'yes', 'programmable', 'dual boiler ', 58),
        (5, 'DeLonghi', 'La Specialista', 'Arte', 'Yellow', 'EC9155.YE', 15, 'vibrating', 'no', '', 'single boiler', 51),
        (6, 'Sage/Breville', 'Barista', 'Express', 'Black truffle', 'SES875BTR2EEU1', 9, 'vibrating', 'yes', 'automatic', 'dual boiler', 54),
    ]
)

## Grinders

For the Grinders table, enter values manually for 5 grinders (info not necessarily accurate, and some fields left empty).

In [43]:
df_grinders = pd.DataFrame(
    columns = [
        "id",
        "manufacturer",
        "model_name",
        "model_name_add",
        "model_specification",
        "product_identifier",
        "operation_type",
        "burr_shape",
        "burr_diameter_mm",
        "burr_material",
        "min_setting",
        "max_setting",
        "min_espresso_range",
        "max_espresso_range",
        "single_dose"
    ],
    data = [
        (1, 'Baratza', 'Encore', 'ESP', 'Black', '495W-230V-F', 'electric', '', pd.NA, '', 0, 40, 0, 20, 'no'), 
        (2, 'Sage/Breville', 'The Smart Grinder', 'Pro', 'Brushed Stainless Steel', 'SCG820BSS4EEU1', 'electric', '', pd.NA, '', 1, 60, 1, 30, 'no'), 
        (3, 'Eureka', 'Mignon', 'Oro', 'Black', 'EAN 8059519339448', 'electric', '', pd.NA, '', 0, 30, 0, 8, 'no'), 
        (4, 'Hario', 'Skerton', 'Plus', 'Schwarz', 'B01LXZACFB', 'manual', '', pd.NA, '', pd.NA, pd.NA, pd.NA, pd.NA, 'no'), 
        (5, 'DeLonghi', 'Dedica', 'Grinder', 'Steel, Silver ', 'KG520.M', 'electric', '', pd.NA, '', 1, 18, pd.NA, pd.NA, 'no')
    ]
)

## Equipment ownership

Assign coffee machines and grinders to different users, which will be stored in EquipmentOwnership table. Match 6 coffee machines and 3 grinders in different combinations (`np.repeat` for machines and `np.tile` for grinders). Add some random purchasing info.

In [44]:
dict_equipment = {
    "id": [],
    "user_id": [],
    "equipment_type": [],
    "coffee_machine_id": [],
    "grinder_id": [],
    "purchase_date": [],
    "purchased_from": [], 
    "purchase_price_eur": []
}

N_COFFEE_MACHINES = 6
N_GRINDERS = 3

dict_equipment["id"] = np.arange(1, N_USERS*2 + 1).tolist()
dict_equipment["user_id"] = np.tile(range(1, N_USERS + 1), 2).tolist()
dict_equipment["equipment_type"] = np.repeat(["coffee_machine", "grinder"], N_USERS).tolist()
dict_equipment["coffee_machine_id"] = np.concatenate((
    np.repeat(range(1, N_COFFEE_MACHINES + 1), N_USERS // N_COFFEE_MACHINES),
    np.repeat(np.nan, N_USERS)
)).tolist()
dict_equipment["grinder_id"] = np.concatenate((
    np.repeat(np.nan, N_USERS), 
    np.tile(range(1, N_GRINDERS + 1), N_USERS // N_GRINDERS)
)).tolist()

for i in range(1, N_USERS*2 + 1):
    date_gen = Datetime(Locale.DE, seed = 42 + i)
    random.seed(42 + i)
    dict_equipment["purchase_date"].append(date_gen.date(start = 2018, end = 2024).strftime("%Y-%m-%d"))
    dict_equipment["purchased_from"].append(random.choice(["Amazon", "eBay", "Mediamarkt", "Roastmarket", "Local Store"]))
    dict_equipment["purchase_price_eur"].append(np.nan)

df_equipment = pd.DataFrame(dict_equipment)
df_equipment["coffee_machine_id"] = df_equipment["coffee_machine_id"].astype(pd.Int64Dtype())
df_equipment["grinder_id"] = df_equipment["grinder_id"].astype(pd.Int64Dtype())

## User defaults

For each user, define a default setup for equipments and preparation details, which will be stored in the UserDefaults table. Coffee machine and grinder align with ownership. Some info constant (basket not pressurized, tamping manual, etc.), some random (wdt, leveler, etc.). 

In [45]:
dict_defaults = {
    "user_id" : [], 
    "coffee_machine_id": [],
    "grinder_id": [],
    "basket_pressurized": [],
    "basket_shot_size": [],
    "portafilter_spout": [],
    "wdt_used": [],
    "tamping_method": [],
    "tamping_weight_kg": [],
    "leveler_used": [],
    "puck_screen_used": [],
    "puck_screen_thickness_mm": [],
    "setup_name": []
}

dict_defaults["user_id"] = np.arange(1, N_USERS + 1).tolist()
dict_defaults["coffee_machine_id"] = np.repeat(range(1, N_COFFEE_MACHINES + 1), N_USERS // N_COFFEE_MACHINES).tolist()
dict_defaults["grinder_id"] = np.tile(range(1, N_GRINDERS + 1), N_USERS // N_GRINDERS).tolist()
dict_defaults["basket_pressurized"] = ["no" for _ in range(N_USERS)]
dict_defaults["basket_shot_size"] = ["double" for _ in range(N_USERS)]
dict_defaults["portafilter_spout"] = ["double" for _ in range(N_USERS)]
np.random.seed(42)
dict_defaults["wdt_used"] = np.random.choice(["yes", "no"], N_USERS, p=[0.5, 0.5]).tolist()
dict_defaults["tamping_method"] = ["manual" for _ in range(N_USERS)]
dict_defaults["tamping_weight_kg"] = [pd.NA for _ in range(N_USERS)]
dict_defaults["leveler_used"] = np.random.choice(["yes", "no"], N_USERS, p=[0.3, 0.7]).tolist()
dict_defaults["puck_screen_used"] = np.random.choice(["yes", "no"], N_USERS, p=[0.5, 0.5]).tolist()
dict_defaults["puck_screen_thickness_mm"] = [pd.NA for _ in range(N_USERS)]
for i in dict_defaults["user_id"]:
    users_name = df_users.loc[df_users["id"] == i, "first_name"].values[0]
    dict_defaults["setup_name"].append(f"{users_name}'s Setup")

df_defaults = pd.DataFrame(dict_defaults)

Normally, additional portafilters are needed for DeLonghi machines to make them non-pressurized; these are defined as bottomless for some variation. Furthermoe, puck screen thickness is given random values if used.

In [46]:
df_defaults.loc[df_defaults["coffee_machine_id"].isin([1, 6]), "portafilter_spout"] = "bottomless"
n_puck_screens = df_defaults["puck_screen_used"].value_counts().get("yes", 0)
df_defaults.loc[df_defaults["puck_screen_used"] == "yes", "puck_screen_thickness_mm"] = np.random.choice([1.0, 1.7, 2.0], n_puck_screens)

## Coffee varieties

This section will generate data for the table CoffeeBeanVarieties. Different logics will be applied for single-origin and blend varieties regarding arabica content, then these subsets will be combined.

### Single origin - arabica

For single origin, intensity will be relatively lower, and never 10/10; acidity always low.

In [47]:
roast_rating_10 = np.array([6, 8, 10])
roast_relative = roast_rating_10 / 10

intensity_rating_10 = np.array([5, 7, 9])
intensity_relative = intensity_rating_10 / 10

acidity_rating_10 = np.array([2])
acidity_relative = acidity_rating_10 / 10

grid = np.array(np.meshgrid(
    roast_relative,
    intensity_relative,
    acidity_relative,
)).T.reshape(-1, 3)

Here single origin will also mean 100% Arabica.

In [48]:
df_variety_so = pd.DataFrame(grid, 
    columns=[
        'roast_level', 
        'intensity', 
        'acidity']
)

df_variety_so["origin_type"] = "single origin"
df_variety_so["arabica_ratio"] = 1.0

Add some random origins, and nonsensical two-word names as if from some Latin American countries.

In [49]:
for i in range(len(df_variety_so)):
    random.seed(42 + i)
    origin_name = random.choice(["Colombia", "Brazil", "Honduras", "Peru", "Mexico", "Gueatemala"])
    df_variety_so.at[i, "origin"] = origin_name

    if origin_name == "Brazil":
        locale = Locale.PT_BR
    else:
        locale = Locale.ES_MX
    
    text_gen = Text(locale, seed=42 + i)
    df_variety_so.at[i, "name"] = f"{text_gen.color()} {text_gen.word()}"

### Blend - arabica-robusta

For blend varieties, intensity is higher and relatively higher acidity is allowed.

In [50]:
roast_rating_10 = np.array([6, 8, 10])
roast_relative = roast_rating_10 / 10

intensity_rating_10 = np.array([8, 10])
intensity_relative = intensity_rating_10 / 10

acidity_rating_10 = np.array([2, 4])
acidity_relative = acidity_rating_10 / 10

grid = np.array(np.meshgrid(
    roast_relative,
    intensity_relative,
    acidity_relative,
)).T.reshape(-1, 3)

Assume there are no 100% Arabica blends for now.

In [51]:
df_variety_blend = pd.DataFrame(grid,
    columns=[
        'roast_level', 
        'intensity', 
        'acidity']
)

df_variety_blend["origin_type"] = "blend"
df_variety_blend["arabica_ratio"] = np.random.choice([0.7, 0.8], len(df_variety_blend), p=[0.4, 0.6])

Add some random origins, and nonsensical two-word names some of them as if from Latin America, and some named like Italian.

In [52]:
for i in range(len(df_variety_blend)):
    random.seed(2025 + i)
    origin_name = random.choice(["Latin America", "Unspecified"])
    df_variety_blend.at[i, "origin"] = origin_name

    if origin_name == "Latin America":
        locale = Locale.ES_MX
    else:
        locale = Locale.IT
    
    text_gen = Text(locale, seed = 2025 + i)
    df_variety_blend.at[i, "name"] = f"{text_gen.color()} {text_gen.word()}"

### Blend - arabica

Relatively dark roast, but not too intense, and low acidity.

In [53]:
roast_rating_10 = np.array([8, 10])
roast_relative = roast_rating_10 / 10

intensity_rating_10 = np.array([6, 8])
intensity_relative = intensity_rating_10 / 10

acidity_rating_10 = np.array([2])
acidity_relative = acidity_rating_10 / 10

grid = np.array(np.meshgrid(
    roast_relative,
    intensity_relative,
    acidity_relative,
)).T.reshape(-1, 3)

These will be 100% Arabica but from different origins. 

In [54]:
df_variety_arbl = pd.DataFrame(grid,
    columns=[
        'roast_level', 
        'intensity', 
        'acidity']
)

df_variety_arbl["origin_type"] = "blend"
df_variety_arbl["arabica_ratio"] = 1.0

Same naming as mixed blends.

In [55]:
for i in range(len(df_variety_arbl)):
    random.seed(101 + i)
    origin_name = random.choice(["Latin America", "Unspecified"])
    df_variety_arbl.at[i, "origin"] = origin_name

    if origin_name == "Latin America":
        locale = Locale.ES_MX
    else:
        locale = Locale.IT
    
    text_gen = Text(locale, seed = 101 + i)
    df_variety_arbl.at[i, "name"] = f"{text_gen.color()} {text_gen.word()}"

### Combined

Combine these three, and add some random made-up roasting house names.

In [56]:
df_variety = pd.concat([df_variety_so, df_variety_blend, df_variety_arbl], ignore_index=True)

df_variety["decaffeinated"] = "no"

producer_list = [
    "Bean Masters", 
    "Coffee Artisans", 
    "Brewed Perfection", 
    "Roast Revolution", 
    "Espresso Excellence"
]

df_variety["producer"] = np.random.choice(producer_list, len(df_variety))

Add some random flavor notes.

In [57]:
flavor_list = [
    "chocolate", 
    "dark chocolate",
    "caramel", 
    "citrus", 
    "berry", 
    "fruity",
    "nutty", 
    "hazelnut",
    "vanilla",
    "floral", 
    "spicy", 
    "smoky"
]

for i in range(len(df_variety)):
    random.seed(42 + i)
    n_flavors = random.randint(1, 3)
    flavors = random.sample(flavor_list, n_flavors)
    df_variety.at[i, "flavor_notes"] = ", ".join(flavors)

Shape the final dataframe.

In [58]:
df_variety = df_variety.reset_index(drop = True)
df_variety = df_variety.reset_index(names = "id")
df_variety["id"] = df_variety["id"] + 1

df_variety = df_variety[[
    "id", 
    "producer", 
    "name",
    "origin",
    "origin_type",
    "arabica_ratio",
    "roast_level",
    "intensity",
    "acidity",
    "flavor_notes", 
    "decaffeinated"
]]

## Coffee purchases

Go through all varieties, assign to a user as purchase, and this will be stored in the table CoffeeBeanPurchases. Add some random data about purchase details.

In [60]:
dict_purchase = {
    "user_id": [],
    "variety_id": [],
    "purchase_date": [],
    "purchased_from": [],
    "weight_kg": [],
    "price_per_kg_eur": []
}

np.random.seed(42)

for i in range(len(df_variety) * 4):
    date_gen = Datetime(Locale.DE, seed = 42 + i)

    user_id = (i % N_USERS) + 1
    variety_id = (i % len(df_variety)) + 1
    purchase_date = date_gen.date(start = 2025).strftime("%Y-%m-%d")
    purchased_from = random.choice(["Amazon", "Roastmarket", "Edeka", "Rewe", "Local roaster"])
    weight_kg = np.random.choice([0.25, 0.5, 1, 1.5], p = [0.3, 0.2, 0.4, 0.1])
    price_per_kg_eur = round(np.random.normal(25, 2), 2)

    dict_purchase["user_id"].append(user_id)
    dict_purchase["variety_id"].append(variety_id)
    dict_purchase["purchase_date"].append(purchase_date)
    dict_purchase["purchased_from"].append(purchased_from)
    dict_purchase["weight_kg"].append(weight_kg)
    dict_purchase["price_per_kg_eur"].append(price_per_kg_eur)

df_purchase = pd.DataFrame(dict_purchase)
df_purchase = df_purchase.reset_index(names = "id")
df_purchase["id"] = df_purchase["id"] + 1

## Espresso

To generate data for the table EspressoExperiments, some made-up relationships are defined here between coffee variety features, extraction parameters, preparation details, and coffee evaluation.

In the first place, extraction ratio is calculated as a function of roast level, grind level, dose, and extraction duration. Below the input values.

In [62]:
roast_rating_10 = np.array([6, 8, 10])
roast_relative = roast_rating_10 / 10

grind_settings_20 = np.array([5, 6, 8, 10, 12, 15, 18])
grind_relative = grind_settings_20 / 20

dose_gr = np.array([16, 17, 17.5, 18, 18.5, 19, 20])
dose_relative = (dose_gr - 15) / 5

extr_sec = np.array([20, 23, 25, 27, 30])
extr_relative = (extr_sec - 20) / 20 + 0.75

Extraction ratio is positively related to the grind size and extraction duration, and negatively with roast level and dose. With arbitrary coefficients and operations, extraction ratio data are generated in a range of 0.75 to 3.75.

In [63]:
grid = np.array(np.meshgrid(roast_relative, grind_relative, dose_relative, extr_relative)).T.reshape(-1, 4)
extr_ratio_raw = 2*grid[:, 1] - grid[:, 0] - grid[:, 2]
extr_ratio_scaled = ((extr_ratio_raw - np.min(extr_ratio_raw)) / (np.max(extr_ratio_raw) - np.min(extr_ratio_raw))) * 2 + 1
extr_ratio_time_adj = extr_ratio_scaled * grid[:, 3]

Turn the resulting numpy array to pandas dataframe for further operations.

In [64]:
baseline_nparray = np.array(np.meshgrid(roast_relative, grind_relative, dose_gr, extr_sec)).T.reshape(-1, 4)
baseline_nparray = np.column_stack((baseline_nparray, extr_ratio_time_adj))
df_baseline = pd.DataFrame(baseline_nparray, columns=['roast', 'grind', 'dose', 'extr_sec', 'extr_ratio'])
for col in df_baseline:
    df_baseline[col] = df_baseline[col].round(2)

Assuming users will try to get an extraction ratio between 1.5 and 2.0, we should get more values in and around this range.

In [18]:
def get_weight_from_extr_ratio(extr_ratio):
    if extr_ratio >= 1.5 and extr_ratio <= 2:
        return 3
    elif extr_ratio < 1.5 and extr_ratio >= 1:
        return 2
    elif extr_ratio > 2 and extr_ratio <= 2.75:
        return 2
    else:
        return 1

df_baseline["sampling_weight"] = df_baseline["extr_ratio"].apply(get_weight_from_extr_ratio)

The below function takes a sample (with replacement) of espressos from the above dataframe according to a selection criteria (e.g. roast level 0.8). It also adds some random variation to extraction ratio and yield. 

In [19]:
def create_coffee_sample(
        baseline = df_baseline,
        selection_col = "roast", 
        selection_crit = 0.8,
        sample_size = 50, 
        weight_col = "sampling_weight",
        random_state = 42,
        extr_ratio_col = "extr_ratio",
        extr_ratio_shift = 0, 
        extr_ratio_sd = 0.1,
        dose_col = "dose",
        yield_shift = 0,
        yield_sd = 1,
):
    new_df = baseline[baseline[selection_col] == selection_crit].sample(
        n = sample_size,
        replace = True,
        weights = weight_col,
        random_state = random_state
    )
    new_df = new_df.drop(columns=[weight_col])
    new_df = new_df.reset_index(drop=True)

    np.random.seed(random_state)

    new_df[extr_ratio_col] = new_df[extr_ratio_col] + np.random.normal(
        loc = extr_ratio_shift,
        scale = extr_ratio_sd,
        size = new_df.shape[0]
    )

    new_df["yield_gr"] = new_df[dose_col] * new_df[extr_ratio_col]
    new_df["yield_gr"] = new_df["yield_gr"] + np.random.normal(
        loc = yield_shift,
        scale = yield_sd,
        size = new_df.shape[0]
    )
    new_df["yield_gr"] = new_df["yield_gr"].round(1)

    new_df = new_df.rename(columns={
        "grind": "grind_setting",
        "dose": "dose_gr",
        "extr_sec": "extraction_time_sec"
    })

    return new_df

Using that function, the below procedure creates a sample of espressos for each coffee purchase corresponding to its roast level. The sample size (number of espressos) depends on the amount of coffee purchased (randomly assigned in the generation of purchase data). The experiment datetime is randomly generated, but fallss within a fortnight of the purchase and is during daytime. The default equipment and preparation setup of the user who purchased the coffee is integrated.

In [20]:
dict_subsets = {}

for purchase_id in df_purchase["id"]:
    variety_id = df_purchase.loc[df_purchase["id"] == purchase_id, "variety_id"].values[0]
    roast_lvl = df_variety.loc[df_variety["id"] == variety_id, "roast_level"].values[0]
    num_espresso = df_purchase.loc[df_purchase["id"] == purchase_id, "weight_kg"].values[0] * 1000 // 20

    df_sub = create_coffee_sample(
        sample_size = num_espresso.astype(int),
        selection_crit = roast_lvl,
        random_state = 100 + purchase_id
    )

    purchase_date = df_purchase.loc[df_purchase["id"] == purchase_id, "purchase_date"].values[0]
    list_coffee_dt = []
    random.seed(42 + purchase_id)
    for i in df_sub.index:
        rd_day = random.randint(1, 14)
        rd_hour = random.randint(0, 9)
        rd_minute = random.randint(0, 59)
        coffee_dt = pd.to_datetime(purchase_date) + pd.to_timedelta(rd_day, unit="d") + pd.to_timedelta(8 + rd_hour, unit="h") + pd.to_timedelta(rd_minute, unit="m")
        list_coffee_dt.append(coffee_dt.strftime("%Y-%m-%d %H:%M:%S"))
    df_sub["experiment_datetime"] = list_coffee_dt

    user_id = df_purchase.loc[df_purchase["id"] == purchase_id, "user_id"].values[0]
    users_defaults = df_defaults.loc[df_defaults["user_id"] == user_id].drop(columns = ["setup_name"])
    for col in users_defaults.columns:
        df_sub[col] = users_defaults[col].values[0]

    dict_subsets[purchase_id] = df_sub

df_coffee_samples = pd.concat(dict_subsets, names=["coffee_bean_purchase_id", "sample_id"]).reset_index()
df_coffee_samples["id"] = df_coffee_samples.index + 1

/tmp/ipykernel_14058/3956816567.py:32: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_coffee_samples = pd.concat(dict_subsets, names=["coffee_bean_purchase_id", "sample_id"]).reset_index()


The functions below create coffee evaluation columns. To calculate values between 1 and 10, they assume a primary relationship between extraction ratio and evaluation, in the form of a convex-down parabola, with the vertex at the coordinates of a moderate extraction ratio and 10. Assuming that more diluted coffees are more tolerable than extremely dense coffees, two parabolae are used for each calculation, meeting at the vertex, and left-hand side being more steep. 

Evaluation of flavor, body, and crema are calculated with slightly varying parameters, and a general evaluation is calculated as a weighted average, giving more weight to flavor, then body. At each stage, some random variation is added.

In [ ]:
def create_eval_cols(df, random_state = 42):
    df["evaluation_general"] = pd.Series(dtype = "float")
    df["evaluation_flavor"] = pd.Series(dtype = "float")
    df["evaluation_body"] = pd.Series(dtype = "float")
    df["evaluation_crema"] = pd.Series(dtype = "float")

    np.random.seed(random_state)

    def calculate_flavor(row):
        if row["extr_ratio"] <= 2:
            val = ((-10)/2.25)*((row["extr_ratio"] - 2)**2) + 10
        else:
            val = ((-10)/9)*((row["extr_ratio"] - 2)**2) + 10
        return val + np.random.normal(0, 0.5)
    
    def calculate_body(row):
        if row["extr_ratio"] <= 1.5:
            val = (-10)*((row["extr_ratio"] - 1.5)**2) + 10
        else:
            val = ((-10)/4)*((row["extr_ratio"] - 1.5)**2) + 10
        return val + np.random.normal(0, 0.5)
    
    def calculate_crema(row):
        if row["extr_ratio"] <= 2.2:
            val = (-10)*((row["extr_ratio"] - 2.2)**2) + 10
        else:
            val = ((-10)/4)*((row["extr_ratio"] - 2.2)**2) + 10
        return val + np.random.normal(0, 0.5)
    
    df["evaluation_flavor"] = df.apply(calculate_flavor, axis=1)
    df["evaluation_body"] = df.apply(calculate_body, axis=1)
    df["evaluation_crema"] = df.apply(calculate_crema, axis=1)

    def calculate_general(row):
        flavor = row["evaluation_flavor"]
        body = row["evaluation_body"]
        crema = row["evaluation_crema"]

        return (flavor*3 + body*2 + crema) / 6 + np.random.normal(0, 0.5)
    
    df["evaluation_general"] = df.apply(calculate_general, axis=1)

    return df

The below function takes into account some extra factors that might influence flavor. First, for coffee varieties, acidity is considered as negative, single origin as positive, and intensity as positive insofar as it only consists of arabica beans. Second, for coffee machines and grinders, the portafilter diameters size and fine (espresso) grinding range are taken as potentially positive contributions, again with some additional random variation.

In [ ]:
def extra_flavor(df, random_state = 42):
    np.random.seed(random_state)

    def variety_effects(row):
        purchase_id = row["coffee_bean_purchase_id"]
        coffee_id = df_purchase.loc[df_purchase["id"] == purchase_id, "variety_id"].values[0]
        origin_type = df_variety.loc[df_variety["id"] == coffee_id, "origin_type"].values[0]
        arabica_ratio = df_variety.loc[df_variety["id"] == coffee_id, "arabica_ratio"].values[0]
        intensity = df_variety.loc[df_variety["id"] == coffee_id, "intensity"].values[0]
        acidity = df_variety.loc[df_variety["id"] == coffee_id, "acidity"].values[0]

        extra_flavor = 0 - acidity 
        if origin_type == "single origin":
            extra_flavor += 0.5
        if arabica_ratio == 1.0:
            extra_flavor += intensity 

        return row["evaluation_flavor"] + extra_flavor + np.random.normal(0, 0.1)
    
    def machine_effects(row):
        machine_id = row["coffee_machine_id"]
        portaf_diam = df_coffee_machines.loc[df_coffee_machines["id"] == machine_id, "portafilter_diam_mm"].values[0]

        grinder_id = row["grinder_id"]
        esp_range_min = df_grinders.loc[df_grinders["id"] == grinder_id, "min_espresso_range"].values[0]
        esp_range_max = df_grinders.loc[df_grinders["id"] == grinder_id, "max_espresso_range"].values[0]
        esp_range = esp_range_max - esp_range_min

        extra_flavor = 0

        if portaf_diam >= 58:
            extra_flavor += 1
        elif portaf_diam >= 54:
            extra_flavor += 0.5
        if esp_range >= 25:
            extra_flavor += 0.5
        elif esp_range >= 15:
            extra_flavor += 0.25

        return row["evaluation_flavor"] + extra_flavor + np.random.normal(0, 0.1)

    df["evaluation_flavor"] = df.apply(variety_effects, axis=1)
    df["evaluation_flavor"] = df.apply(machine_effects, axis=1)
    df["evaluation_general"] = df[["evaluation_flavor", "evaluation_body", "evaluation_crema"]].mean(axis=1)

    return df

The next function scales the values to a range of 1 to 10, and rounds up to the closest integer. Before rounding up, increases or decreases half of the values by 0.5, to mix up adjacent categories, hence to add some more noise to the data (evaluation is supposed to be quite subjective and volatile).

In [ ]:
def scale_noise_int(df):
    fl_min = df["evaluation_flavor"].min()
    fl_max = df["evaluation_flavor"].max()
    bd_min = df["evaluation_body"].min()
    bd_max = df["evaluation_body"].max()
    cr_min = df["evaluation_crema"].min()
    cr_max = df["evaluation_crema"].max()
    gen_min = df["evaluation_general"].min()
    gen_max = df["evaluation_general"].max()

    noise = {}
    for i in range(4):
        noise[i] = np.random.choice([-0.5, 0, 0.5], df.shape[0], p = [0.25, 0.5, 0.25])

    df["evaluation_flavor"] = ((df["evaluation_flavor"] - fl_min) / (fl_max - fl_min)) * 10 + noise[0]
    df["evaluation_body"] = ((df["evaluation_body"] - bd_min) / (bd_max - bd_min)) * 10 + noise[1]
    df["evaluation_crema"] = ((df["evaluation_crema"] - cr_min) / (cr_max - cr_min)) * 10 + noise[2]
    df["evaluation_general"] = ((df["evaluation_general"] - gen_min) / (gen_max - gen_min)) * 10 + noise[3]

    df["evaluation_flavor"] = df["evaluation_flavor"].round(0).clip(1, 10).astype(int)
    df["evaluation_body"] = df["evaluation_body"].round(0).clip(1, 10).astype(int)
    df["evaluation_crema"] = df["evaluation_crema"].round(0).clip(1, 10).astype(int)
    df["evaluation_general"] = df["evaluation_general"].round(0).clip(1, 10).astype(int)

    return df

Applying these three functions, the dataframe is extended with evaluation columns:

In [22]:
df_espresso_samples = create_eval_cols(df_coffee_samples.copy())
df_espresso_samples = extra_flavor(df_espresso_samples.copy())
df_espresso_samples = scale_noise_int(df_espresso_samples.copy())

With addition of final missing columns, and keeping only the necessary ones in the proper order, the final dataframe is ready:

In [34]:
df_espresso_samples["water_temp_c"] = 93
df_espresso_samples["evaluation_notes"] = ""

df_espresso = df_espresso_samples[[
    "id", 
    "experiment_datetime",
    "user_id", 
    "coffee_machine_id", 
    "grinder_id", 
    "basket_pressurized",
    "basket_shot_size",
    "portafilter_spout",
    "wdt_used",
    "tamping_method",
    "tamping_weight_kg",
    "leveler_used",
    "puck_screen_used",
    "puck_screen_thickness_mm",
    "coffee_bean_purchase_id",
    "grind_setting",
    "dose_gr",
    "water_temp_c",
    "extraction_time_sec",
    "yield_gr",
    "evaluation_general", 
    "evaluation_flavor",
    "evaluation_body", 
    "evaluation_crema",
    "evaluation_notes"
]]

## Write dataframes to the database

Finally, these 8 dataframes can be written to the database. It is assumed here that the database with corresponding schema exists, and there is a known user with at least appending privileges (`INSERT INTO`). 

Libraries for database connection, and reading info from dotenv file.

In [31]:
from dotenv import load_dotenv
import os
from sqlmodel import create_engine

Set up database connection:

In [32]:
load_dotenv()

driver = os.getenv("DB_DRIVER")
connector = os.getenv("DB_CONNECTOR")
user = os.getenv("DB_USERNAME")
password = os.getenv("DB_PASSWORD")
host = os.getenv("DB_HOST")
port = os.getenv("DB_PORT")
db_name = os.getenv("DB_NAME")

db_path = f"{driver}+{connector}://{user}:{password}@{host}:{port}/{db_name}"
db_engine = create_engine(db_path, echo = False)

Send data to the database:

In [ ]:
df_users.to_sql("Users", db_engine, if_exists = "append", index = False)
df_coffee_machines.to_sql("CoffeeMachines", db_engine, if_exists = "append", index = False)
df_grinders.to_sql("Grinders", db_engine, if_exists = "append", index = False)
df_equipment.to_sql("EquipmentOwnership", db_engine, if_exists = "append", index = False)
df_defaults.to_sql("UserDefaults", db_engine, if_exists = "append", index = False)
df_variety.to_sql("CoffeeBeanVarieties", db_engine, if_exists = "append", index = False)
df_purchase.to_sql("CoffeeBeanPurchases", db_engine, if_exists = "append", index = False)
df_espresso.to_sql("EspressoExperiments", db_engine, if_exists = "append", index = False)